In [ ]:
%%python
import ROOT

In [ ]:
gSystem->Load("libQuantLib.so")

In [ ]:
#include <ql/quantlib.hpp>
#include <sstream>

In [ ]:
using namespace QuantLib;
namespace ql = QuantLib;

In [ ]:
class Pricer {
    private:
       Date todaysDate;
       Date settlementDate;
       Date maturity;
       Real stock;
       Real strike;
       Spread dividendYield;
       Rate riskFreeRate;
       Volatility volatility;
       Option::Type type;
public:    
    Pricer() : todaysDate(27, Jan, 2011),
    settlementDate(27, Jan, 2011),
    maturity(27, May, 2011),
    stock(47.0),
    strike(40.0),
    dividendYield(0.00),
    riskFreeRate(0.05),
    volatility(0.20),
    type(Option::Call) {
    };
    void input(const std::string &type_,
              int dayToday,
              int monthToday,
              int yearToday,
              int dayMaturity,
              int monthMaturity,
              int yearMaturity,
              Real stock_,
              Real strike_) {
        if (type_ == "Call") {
            type = Option::Call;
        } else {
            type = Option::Put;
        }
        todaysDate = Date(static_cast<Day>(dayToday), 
                          static_cast<Month>(monthToday), 
                          static_cast<Year>(yearToday));
        maturity = Date(static_cast<Day>(dayMaturity), 
                        static_cast<Month>(monthMaturity), 
                        static_cast<Year>(yearMaturity));
        stock = stock_;
        strike = strike_;
    };
std::string output() {
    std::stringstream s;
// date set up
Calendar calendar = TARGET();
Settings::instance().evaluationDate() = todaysDate;

// option parameters
DayCounter dayCounter = Actual365Fixed();

std::shared_ptr<Exercise> europeanExercise(new EuropeanExercise(maturity));

Handle<Quote> underlyingH(std::shared_ptr<Quote>(new SimpleQuote(stock)));

// bootstrap the yield/dividend/vol curves
Handle<YieldTermStructure> flatTermStructure(ext::shared_ptr<YieldTermStructure>(
	new FlatForward(
	settlementDate,
	riskFreeRate,
	dayCounter)));

Handle<YieldTermStructure> flatDividendTS(ext::shared_ptr<YieldTermStructure>(
	new FlatForward(settlementDate,
	dividendYield,
	dayCounter)));

Handle<BlackVolTermStructure> flatVolTS(ext::shared_ptr<BlackVolTermStructure>(
	new BlackConstantVol(
	settlementDate,
	calendar,
	volatility,
	dayCounter)));

ext::shared_ptr<StrikedTypePayoff> payoff(
	new PlainVanillaPayoff(
	type,
	strike));

ext::shared_ptr<BlackScholesMertonProcess> bsmProcess(
	new BlackScholesMertonProcess(
	underlyingH,
	flatDividendTS,
	flatTermStructure,
	flatVolTS));

// our option is European-style
VanillaOption europeanOption(
	payoff,
	europeanExercise);

// computing the option price with the analytic Black-Scholes formulae
europeanOption.setPricingEngine(ext::shared_ptr<PricingEngine>(
	new AnalyticEuropeanEngine(
	bsmProcess)));

// outputting
s << "Option type = " << type << std::endl;
s << "Maturity = " << maturity << std::endl;
s << "Stock price = " << stock << std::endl;
s << "Strike = " << strike << std::endl;
s << "Risk-free interest rate = " << io::rate(riskFreeRate) << std::endl;
s << "Dividend yield = " << io::rate(dividendYield) << std::endl;
s << "Volatility = " << io::volatility(volatility) << std::endl << std::endl;
s <<"European Option value = " << europeanOption.NPV() << std::endl;
    return s.str();
}
};


In [ ]:
QuantLib::Date get_date(int day, int month, int year) {
    return QuantLib::Date(ql::Day(day), ql::Month(month), ql::Year(year));
}

In [ ]:
auto b = Pricer();
QuantLib::Date maturity;
QuantLib::Date evaluationDate;

In [ ]:
auto d = get_date(5, 12, 2020);

In [ ]:
std::cout << d;

In [ ]:
c = b.output()

In [ ]:
%%python
import ipywidgets
import ROOT

type = ipywidgets.Dropdown(
    options=['Call', 'Put'],
    value='Call',
    description='Type:',
    disabled=False,
)


maturity = ipywidgets.widgets.DatePicker(
    description='Maturity',
    disabled=False
)
evaluation = ipywidgets.widgets.DatePicker(
    description='Evaluation date',
    disabled=False
)

stock_price = ipywidgets.FloatLogSlider(description="Stock Price",
                                       min=-2, max=4)
strike = ipywidgets.FloatLogSlider(description="Strike",
                                  min=-2,max=4)
risk_free = ipywidgets.FloatSlider(description="Risk Free")
dividend = ipywidgets.FloatSlider(description="Dividend")
volatility = ipywidgets.FloatSlider(description="Volatility")

button = ipywidgets.Button(description="Click Me!")
output = ipywidgets.Output();

def on_button_clicked(button):
    q = maturity.value
    p = evaluation.value
    ROOT.b.input(
        type.value,
        p.day, p.month, p.year,
                q.day, q.month, q.year,
                stock_price.value, strike.value)
    output.append_stdout(ROOT.b.output())
#    output.append.stdout(ROOT.maturity)

button.on_click(on_button_clicked)

In [ ]:
%%python
import datetime
from dateutil.relativedelta import relativedelta
evaluation.value = datetime.date.today()
maturity.value = datetime.date.today() + \
   relativedelta(months=+6)
a = ipywidgets.VBox([
    type,
                maturity,
                evaluation,
                stock_price,
                strike,
                risk_free,
                dividend,
                volatility,
                button, output])
a